# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [239]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
import pandas as pd
import numpy as np
import glob
import os
from IPython.display import display
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [101]:
path = os.path.join(DATA_FOLDER, 'ebola')
guinea_folder = os.path.join(path, 'guinea_data')
liberia_folder = os.path.join(path, 'liberia_data')
sl_folder = os.path.join(path, 'sl_data')
guinea_files = glob.glob(os.path.join(guinea_folder, "*.csv"))
liberia_files = glob.glob(os.path.join(liberia_folder, "*.csv"))
sl_files = glob.glob(os.path.join(sl_folder, "*.csv"))

tmp_df = pd.read_csv(guinea_files[0], usecols=["Date", "Description", "Totals"], parse_dates=['Date'])
tmp_df[tmp_df.Description.str.contains('new') | tmp_df.Description.str.contains('Total deaths')]
df = pd.read_csv(guinea_files[2], parse_dates=['Date'])
df = df.fillna(0)
# df.loc[:, df.columns != 'Date' &]
df = df[(df.Description.str.contains('new cases', case=False) | df.Description.str.contains('deaths', case=False))
   & ~df.Description.str.contains('health workers')]
print(df.loc[:, (df.columns != 'Date') & (df.columns != 'Description') & (df.columns != 'Totals')].astype(int).sum(axis=1))
df

0       5
1       0
2      14
3      19
9       3
10      5
11    162
12    468
13    635
dtype: int64


,Date,Description,Totals,Conakry,Gueckedou,Macenta,Dabola,Kissidougou,Dinguiraye,Telimele,...,Mzerekore,Yomou,Dubreka,Forecariah,Kerouane,Coyah,Dalaba,Beyla,Kindia,Lola
0,2014-09-22,New cases of suspects,5,3,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
1,2014-09-22,New cases of probables,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,2014-09-22,New cases of confirmed,14,3,2,7,0,0,0,0,...,0,0,0,0,0,2,0,0.0,0.0,0.0
3,2014-09-22,Total new cases registered so far,19,6,4,7,0,0,0,0,...,0,0,0,0,0,2,0,0.0,0.0,0.0
9,2014-09-22,New deaths registered,3,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
10,2014-09-22,Total deaths of suspects,5,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
11,2014-09-22,Total deaths of probables,162,8,111,12,3,5,1,3,...,11,0,0,0,0,0,0,0.0,0.0,0.0
12,2014-09-22,Total deaths of confirmed,468,58,183,161,1,2,0,7,...,9,5,7,7,5,5,0,0.0,0.0,0.0
13,2014-09-22,Total deaths (confirmed + probables + suspects),635,68,295,175,4,7,1,10,...,20,5,7,7,5,5,0,0.0,0.0,0.0


In [214]:
path = os.path.join(DATA_FOLDER, 'ebola')

guinea_folder = os.path.join(path, 'guinea_data')
liberia_folder = os.path.join(path, 'liberia_data')
sl_folder = os.path.join(path, 'sl_data')

guinea_files = glob.glob(os.path.join(guinea_folder, "*.csv"))
liberia_files = glob.glob(os.path.join(liberia_folder, "*.csv"))
sl_files = glob.glob(os.path.join(sl_folder, "*.csv"))
list_df = []
for file in liberia_files:
    df = pd.read_csv(file, usecols=["Date", "Variable", "National"], parse_dates=['Date'])
#     df.Variable.replace('\s+', '_',regex=True,inplace=True)
    list_df.append(df)
df = pd.concat(list_df)
df[(df.Date.dt.month == 8)]
# df_sum = df.groupby([(df.Date.dt.year),(df.Date.dt.month),(df.Variable)]).sum()
# df_sum
# df_sum[np.in1d(df_sum.index.get_level_values(2),['Total death/s in confirmed, probable, suspected cases','Total death/s in confirmed, \n probable, suspected cases'])]
# df_sum[np.in1d(df_sum.index.get_level_values(2),['Total_death/s_in_confirmed,_probable,_suspected_cases'])]

,Date,Variable,National
0,2014-08-25,Specimens collected,4.0
1,2014-08-25,Specimens pending for testing,38.0
2,2014-08-25,Total specimens tested,NaN
3,2014-08-25,Newly reported deaths,28.0
4,2014-08-25,Total death/s in confirmed cases,280.0
5,2014-08-25,Total death/s in probable cases,309.0
6,2014-08-25,Total death/s in suspected cases,182.0
7,2014-08-25,"Total death/s in confirmed, probable, suspecte...",771.0
8,2014-08-25,Case Fatality Rate (CFR) - Confirmed & Probabl...,NaN
9,2014-08-25,Newly reported contacts,151.0


In [234]:
path = os.path.join(DATA_FOLDER, 'ebola')

guinea_folder = os.path.join(path, 'guinea_data')
liberia_folder = os.path.join(path, 'liberia_data')
sl_folder = os.path.join(path, 'sl_data')

guinea_files = glob.glob(os.path.join(guinea_folder, "*.csv"))
liberia_files = glob.glob(os.path.join(liberia_folder, "*.csv"))
sl_files = glob.glob(os.path.join(sl_folder, "*.csv"))

list_df = []
for file in guinea_files:
    df = pd.read_csv(file, usecols=["Date", "Description", "Totals"], parse_dates=['Date'])
    df = df.fillna(0)
    # df.loc[:, df.columns != 'Date' &]
    df = df[(df.Description.str.contains('new cases', case=False)
             | df.Description.str.contains('deaths', case=False)
             | df.Description.str.contains('Cumulative'))
       & ~df.Description.str.contains('health workers')]
    list_df.append(df)
    df.Totals = df.Totals.astype(int)
    df['Country'] = 'Guinea'
    
    
df = pd.concat(list_df)
df[(df.Date.dt.month == 9) & ((df.Description == 'Total new cases registered so far') | (df.Description == 'Cumulative (confirmed + probable + suspects)'))].sort_values(by=['Date', 'Description'])
# df.groupby([(df.Country), (df.Date.dt.year),(df.Date.dt.month),(df.Description)]).sum()
# df.Date.dt.year
# df = pd.read_csv(guinea_files[4], usecols=["Date", "Description", "Totals"], parse_dates=['Date'])
# df = df.fillna(0)
# # df.loc[:, df.columns != 'Date' &]
# df = df[(df.Description.str.contains('new cases', case=False) | df.Description.str.contains('deaths', case=False))
#    & ~df.Description.str.contains('health workers')]
# df

,Date,Description,Totals,Country
7,2014-09-02,Cumulative (confirmed + probable + suspects),790,Guinea
3,2014-09-02,Total new cases registered so far,25,Guinea
7,2014-09-04,Cumulative (confirmed + probable + suspects),823,Guinea
3,2014-09-04,Total new cases registered so far,30,Guinea
7,2014-09-07,Cumulative (confirmed + probable + suspects),861,Guinea
3,2014-09-07,Total new cases registered so far,16,Guinea
7,2014-09-08,Cumulative (confirmed + probable + suspects),861,Guinea
3,2014-09-08,Total new cases registered so far,16,Guinea
7,2014-09-09,Cumulative (confirmed + probable + suspects),867,Guinea
3,2014-09-09,Total new cases registered so far,16,Guinea


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [237]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

## Answer
### Q1
- 

In [271]:
df = pd.read_excel(os.path.join(DATA_FOLDER,'titanic.xls'))
df['pclass'] = df.pclass.astype('category')
df['survived'] = df.survived.astype('category')
df['sex'] = df.sex.astype('category')


0